In [1]:
import os
from azureml.core import Model, Workspace
from azureml.core.authentication import InteractiveLoginAuthentication
from azureml.core.compute import AksCompute, ComputeTarget
from azureml.core.environment import Environment
from azureml.core.experiment import Experiment
from azureml.core.model import InferenceConfig
from azureml.core.webservice import LocalWebservice, AciWebservice, AksWebservice
from azureml.pipeline.wrapper import PipelineRun

### Prepare your workspace

In [2]:
tenant_id = "72f988bf-86f1-41af-91ab-2d7cd011db47"
InteractiveLoginAuthentication(tenant_id=tenant_id)
workspace = Workspace.from_config('config.json')
print(workspace.name, workspace.resource_group, workspace.location, workspace.subscription_id,
      workspace.compute_targets.keys(), sep='\n')

fundamental3
fundamental
eastasia
4f455bd0-f95a-4b7d-8d08-078611508e0b
dict_keys(['myaks2', 'aml-compute', 'my-compute', 'compute-deploy'])


### Get a list of experiment names from the workspace

In [3]:
exp_name_list = [exp.name for exp in Experiment.list(workspace)]
exp_name_list

['fasttext_test',
 'sample-pipelines',
 'automobile',
 'fasttext_predict',
 'sample-pipelines2',
 'fasttext_with_two_training_process',
 'train-within-notebook',
 'train-on-local',
 'logging-api-test',
 'fasttext_with_one_training_process',
 'fasttext_train',
 'my_test',
 'split_data_txt',
 'compare_two_models',
 'yucongj-test',
 'fasttext_parallel_score',
 'parallel',
 'dir',
 'test0717',
 'test_0727',
 'test_0727_experiment',
 'localtest',
 'mpi_0729',
 'mpi_0729_experiment',
 'test',
 'para_0729',
 'para_0729_experiment',
 'basic_0721',
 'basic_0721_experiment',
 'deploy',
 'fasttext_training_process',
 'fasttext_batch_inference']

### Choose the experiment you want with its name

In [4]:
experiment_name = "fasttext_training_process"
experiment = Experiment(workspace, experiment_name)
experiment

Name,Workspace,Report Page,Docs Page
fasttext_training_process,fundamental3,Link to Azure Machine Learning studio,Link to Documentation


In [5]:
# azureml.pipeline.core.run.PipelineRun
run = experiment.get_runs().__next__()
run

Experiment,Id,Type,Status,Details Page,Docs Page
fasttext_training_process,ee7d355b-48e2-46a1-a803-f878ff7a549f,azureml.PipelineRun,Completed,Link to Azure Machine Learning studio,Link to Documentation


### Get a PipelineRun object

In [6]:
run_id = run.id
# azureml.pipeline.wrapper._pipeline_run.PipelineRun
pipeline_run = PipelineRun(experiment, run_id)
pipeline_run

Experiment,Id,Type,Status,Details Page,Docs Page
fasttext_training_process,ee7d355b-48e2-46a1-a803-f878ff7a549f,azureml.PipelineRun,Completed,Link to Azure Machine Learning studio,Link to Documentation


### Visualize the pipeline

In [7]:
# To do

In [8]:
# name will be from the visualization result.
step_run_list = pipeline_run.PipelineRun_find_step_run(name='FastText Train')
for s in step_run_list:
    print(s,'\n\n')

StepRun(Experiment: fasttext_training_process,
Id: a0ffbff0-8961-4281-87b2-83459dcacc62,
Type: azureml.StepRun,
Status: Completed) 


StepRun(Experiment: fasttext_training_process,
Id: df736570-dd90-4eda-a6d6-048f7e30ccdb,
Type: azureml.StepRun,
Status: Completed) 




### Get a StepRun object

In [33]:
# step_run_id will be from the visualization result.
step_run_id = '530c79d5-3304-4cbe-92af-c45ce31fe98b'
step_run = pipeline_run.get_step_run(step_run_id)
step_run

Experiment,Id,Type,Status,Details Page,Docs Page
fasttext_training_process,530c79d5-3304-4cbe-92af-c45ce31fe98b,azureml.StepRun,Completed,Link to Azure Machine Learning studio,Link to Documentation


### Download the trained model from azure storage

In [34]:
# name will be from the visualization result.
# get_port() should supports three kinds of names: (1)the_better_model (2)The better model (3)The_better_model
port = step_run.get_port(name='The_better_model')
saved_path = port.download(overwrite=True)
print('model save at: {}'.format(saved_path))

Downloaded azureml/530c79d5-3304-4cbe-92af-c45ce31fe98b/The_better_model/label.txt, 1 files out of an estimated total of 3
Downloaded azureml/530c79d5-3304-4cbe-92af-c45ce31fe98b/The_better_model/word_to_index.json, 2 files out of an estimated total of 3
Downloaded azureml/530c79d5-3304-4cbe-92af-c45ce31fe98b/The_better_model/BestModel, 3 files out of an estimated total of 3
model save at: /tmp/azureml/530c79d5-3304-4cbe-92af-c45ce31fe98b/The_better_model


### Register the trained model for deployment

In [35]:
model_path = saved_path
model = Model.register(workspace, model_path=saved_path, model_name='model_for_deployment')
model

Registering model model_for_deployment


Model(workspace=Workspace.create(name='fundamental3', subscription_id='4f455bd0-f95a-4b7d-8d08-078611508e0b', resource_group='fundamental'), name=model_for_deployment, id=model_for_deployment:6, version=6, tags={}, properties={})

### Get an Environment object

In [36]:
env_list = Environment.list(workspace)
name = 'env_for_deployment'
if name not in env_list:
    file_path = 'deployment/env_for_deployment.yaml'
    env = Environment.from_conda_specification(name=name, file_path=file_path)
    env = env.register(workspace=workspace)
else:
    env = Environment.get(workspace=workspace, name=name)

### Define inference configuration

In [37]:
entry_script = 'scoring_for_deployment.py'
source_directory = 'deployment'
inference_config = InferenceConfig(entry_script=entry_script, source_directory=source_directory,
                                   environment=env)

### Deploy locally

In [53]:
service_name = 'local-deploy-test'
models = [model]
port = 8892
deployment_config = LocalWebservice.deploy_configuration(port=port)
service_locally = Model.deploy(workspace=workspace, name=service_name, models=models, inference_config=inference_config,
                               deployment_config=deployment_config)
service_locally.wait_for_deployment(show_output=True)
print(service_locally.state)
print(service_locally.get_logs())

Generating Docker build context.
Package creation Succeeded
Logging into Docker registry fundamental33c005c1f.azurecr.io
Logging into Docker registry fundamental33c005c1f.azurecr.io
Building Docker image from Dockerfile...
Step 1/5 : FROM fundamental33c005c1f.azurecr.io/azureml/azureml_ac0df2cc6dbad51226a8d176555b98dc
 ---> 0b06c2e718e3
Step 2/5 : COPY azureml-app /var/azureml-app
 ---> e495da6198c5
Step 3/5 : RUN mkdir -p '/var/azureml-app' && echo eyJhY2NvdW50Q29udGV4dCI6eyJzdWJzY3JpcHRpb25JZCI6IjRmNDU1YmQwLWY5NWEtNGI3ZC04ZDA4LTA3ODYxMTUwOGUwYiIsInJlc291cmNlR3JvdXBOYW1lIjoiZnVuZGFtZW50YWwiLCJhY2NvdW50TmFtZSI6ImZ1bmRhbWVudGFsMyIsIndvcmtzcGFjZUlkIjoiYTdjMmFjYWEtYzhmMS00NDhiLWI4OTQtYzJlN2E3MWIzYTMyIn0sIm1vZGVscyI6e30sIm1vZGVsc0luZm8iOnt9fQ== | base64 --decode > /var/azureml-app/model_config_map.json
 ---> Running in 3b1e52cc7257
 ---> 25d42146b8ac
Step 4/5 : RUN mv '/var/azureml-app/tmpjbqkb0gk.py' /var/azureml-app/main.py
 ---> Running in 2d4da5272666
 ---> 2e66c544544f
Step 5/5 : CMD 

### Deploy to ACI (Azure Container Instances)
- Every time we deploy to ACI, we need to change the service_name or we delete the existing service beforehand.
- To delete the existing service, let overwrite be True.

In [39]:
service_name = 'aci-deploy-test'
models = [model]
deployment_config = AciWebservice.deploy_configuration(cpu_cores=1, memory_gb=1)
service_aci = Model.deploy(workspace, service_name, models=models, inference_config=inference_config,
                           deployment_config=deployment_config, overwrite=True)
service_aci.wait_for_deployment(show_output=True)
print(service_aci.state)

Running.......
Succeeded
ACI service creation operation finished, operation "Succeeded"
Healthy


### Deploy to AKS (Azure Kubernetes Service)

In [40]:
aks_subscription_id = '74eccef0-4b8d-4f83-b5f9-fa100d155b22'
aks_resource_group = 'DesignerDRI'
aks_workspace_name = 'DesignerDRI_EASTUS'
aks_workspace = Workspace(aks_subscription_id, aks_resource_group, aks_workspace_name)

In [41]:
# register model in this workspace
model = Model.register(aks_workspace, model_path=model_path, model_name='model_for_deploy', tags={'deploy': 1})

Registering model model_for_deploy


In [42]:
# load aks_target
name = 'attached-aks'
compute_target_name_list = [target.name for target in AksCompute.list(aks_workspace)]
if not name in compute_target_name_list:
    aks_resource_group = 'AmlStudioV2DRI'
    cluster_name = 'aks-dev-6node33512023a'
    attach_config = AksCompute.attach_configuration(resource_group=aks_resource_group, cluster_name=cluster_name)
    aks_target = ComputeTarget.attach(aks_workspace, name, attach_config)
    aks_target.wait_for_completion(show_output=True)
else:
    aks_target = AksCompute(aks_workspace, name)
aks_target = AksCompute(aks_workspace, name)
aks_target

AksCompute(workspace=Workspace.create(name='DesignerDRI_EASTUS', subscription_id='74eccef0-4b8d-4f83-b5f9-fa100d155b22', resource_group='DesignerDRI'), name=attached-aks, id=/subscriptions/74eccef0-4b8d-4f83-b5f9-fa100d155b22/resourceGroups/DesignerDRI/providers/Microsoft.MachineLearningServices/workspaces/DesignerDRI_EASTUS/computes/attached-aks, type=AKS, provisioning_state=Succeeded, location=eastus, tags=None)

### deploy to AKS (Azure Kubernetes Service)
- every time we deploy to AKS, we need to change the service_name or we delete the existing service beforehand

In [43]:
# deploy to AKS (Azure Kubernetes Service)
# every time we deploy to AKS, we need to change the service_name or we delete the existing service beforehand
service_name = 'aks-deploy-test'
services = aks_workspace.webservices
if service_name in services:
    services[service_name].delete()
models = [model]
token_auth_enabled = True
# Only one type of Auth may be enabled
if token_auth_enabled:
    # key auth
    auth_enabled = False
else:
    auth_enabled = True

deployment_config = AksWebservice.deploy_configuration(cpu_cores=1, memory_gb=1,
                                                       token_auth_enabled=token_auth_enabled,
                                                       auth_enabled=auth_enabled)
service_aks = Model.deploy(aks_workspace, service_name, models, inference_config, deployment_config, aks_target)
service_aks.wait_for_deployment(show_output=True)
print(service_aks.state)

Running...
Succeeded
AKS service creation operation finished, operation "Succeeded"
Healthy


### Consume the service

In [44]:
### Get a StepRun object# consume
import requests
import json
from azureml.core.authentication import InteractiveLoginAuthentication

# Get a token to authenticate to the compute instance from remote
interactive_auth = InteractiveLoginAuthentication()
auth_header = interactive_auth.get_authentication_header()

# Create and submit a request using the auth header
headers = auth_header
# Add content type header
headers.update({'Content-Type': 'application/json'})
# print(headers)


standard_sample_input = {'param': {'input_sentence': '受疫情影响, 今年很多学生不得不在家上课'}}
standard_sample_input = json.dumps(standard_sample_input)

service = service_locally
# service = service_aci

# service = service_aks
# token, refresh_by = service.get_token()
# headers['Authorization']=f'Bearer {token}'


response = requests.post(service.scoring_uri, data=standard_sample_input, headers=headers)
print(service.scoring_uri)
print(response)
# print(response.status_code)
# print(response.elapsed)
print(response.content)
print(response.json())

http://localhost:8892/score
<Response [500]>
b"[Errno 2] No such file or directory: '\xe5\x8f\x97\xe7\x96\xab\xe6\x83\x85\xe5\xbd\xb1\xe5\x93\x8d, \xe4\xbb\x8a\xe5\xb9\xb4\xe5\xbe\x88\xe5\xa4\x9a\xe5\xad\xa6\xe7\x94\x9f\xe4\xb8\x8d\xe5\xbe\x97\xe4\xb8\x8d\xe5\x9c\xa8\xe5\xae\xb6\xe4\xb8\x8a\xe8\xaf\xbe'"


JSONDecodeError: Expecting value: line 1 column 2 (char 1)